In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [ ]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

In [10]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_threshold = [1,2,3,4,5]

In [ ]:
list_configs=[]
for lt in lost_thresholds:
    for config in list_dicts:
        current_config_dict = config.copy()
        parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt}
        flat_dict = flatten_dict(current_config_dict)
        flat_dict.update(parameter_map)
        updated_config_dict = expand_dict(flat_dict)
        file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}.json".format(len(list_configs), 
                                            current_config_dict['video_config']['video_id'], lt))
        list_configs.append(file_name)
        with open(file_name, 'w') as f:
            json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

In [ ]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-1-2", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-1-2",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

In [13]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [ ]:
experiment.stage_experiment()

In [3]:
experiment.launch_experiment()

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/Users/jason.katz/Downloads/search_results2.csv')

In [4]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [5]:
output_notebook()

Loading BokehJS ...

In [ ]:
data['lt'] = 1
for index, row in data.iterrows():
    data['lt'][index] = row['job_id'][-2:]

In [ ]:
data['game'] = 'id'
for index, row in data.iterrows():
    data['game'][index] = row['job_id'][16:22]

In [10]:
scat1 = Scatter(data, x='lt', y='mota', color='game')
show(scat1)

In [11]:
data_lt = data.copy()
data_lt['lost_threshold'] = data_lt['lt']
data_lt = data_lt.groupby(['lt']).mean()
data_lt

,mota,motp,motr,mostly_tracked,mt_auc,lost_threshold
lt,,,,,,
1,0.735035,0.640477,0.819166,0.666049,0.809022,1
2,0.741263,0.638814,0.837608,0.709025,0.826910,2
3,0.743281,0.637822,0.850431,0.734457,0.839622,3
4,0.744797,0.636943,0.860527,0.750982,0.849614,4
5,0.744160,0.636579,0.866801,0.763067,0.855742,5


In [12]:
scat2 = Scatter(data_lt, x = 'lost_threshold', y='mota')
scat3 = Scatter(data_lt, x = 'lost_threshold', y='motp')
scat4 = Scatter(data_lt, x = 'lost_threshold', y='mostly_tracked')
scat5 = Scatter(data_lt, x = 'lost_threshold', y='mt_auc')
from bokeh.layouts import column, row
show(column(row(scat2, scat3), row(scat4, scat5)))